# 用Dense将mnist数据集跑到99%

## 初始化，使用tf.v1，关掉debug信息

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


## 设置超参数

In [2]:
learning_rate = 0.001
num_steps = 600*5
batch_size = 100
display_step = 100
validation_step = 600

num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)


## 加载mnist数据

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


x_train = np.reshape(x_train,(-1,784))
x_test = np.reshape(x_test,(-1,784))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


## 验证数据

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
image_index = 7777 # You may select anything up to 60,000
print(y_train[image_index]) # The label is 8
#plt.imshow(x_train[image_index].reshape((28,28)), cmap='Greys')
#plt.show()

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


## 设置训练batch generator

In [5]:
def train_generator(batch_size):
    while True:
        for i in range(0, x_train.shape[0], batch_size):
            yield x_train[i:i+batch_size,:], y_train[i:i+batch_size]
        
def test_generator(batch_size):
    while True:
        for i in range(0, x_test.shape[0], batch_size):
            yield x_test[i:i+batch_size,:], y_test[i:i+batch_size]


## 输入变量Placeholder

In [6]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


# 神经网络部分
## 卷积层

In [7]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')



## 核心函数

In [8]:
def conv_net(x, weights, biases, dropout):
    fc1 = tf.add(tf.matmul(x, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## 存储各个变量

In [9]:
weights = {
    'wd1': tf.Variable(tf.random_normal([784, 128])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([128, num_classes]))
}

biases = {
    'bd1': tf.Variable(tf.zeros([128])),
    'out': tf.Variable(tf.zeros([num_classes]))
}


# 训练和评估
## 各个评估变量

In [10]:
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## 训练部分

In [11]:
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next(train_generator(batch_size))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.2})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 0.2})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        if step % validation_step == 0:
            print("Testing Accuracy:", \
                    sess.run(accuracy, feed_dict={X: x_test[:batch_size,:],
                        Y: y_test[:batch_size],
                                              keep_prob: 1.0}))
    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: x_test,
                Y: y_test,
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 209.0592, Training Accuracy= 0.110
Step 100, Minibatch Loss= 115.5005, Training Accuracy= 0.170
Step 200, Minibatch Loss= 50.3314, Training Accuracy= 0.290
Step 300, Minibatch Loss= 30.4576, Training Accuracy= 0.350
Step 400, Minibatch Loss= 15.8365, Training Accuracy= 0.430
Step 500, Minibatch Loss= 6.5537, Training Accuracy= 0.620
Step 600, Minibatch Loss= 4.9927, Training Accuracy= 0.600
Testing Accuracy: 0.56
Step 700, Minibatch Loss= 4.1845, Training Accuracy= 0.670
Step 800, Minibatch Loss= 2.8545, Training Accuracy= 0.630
Step 900, Minibatch Loss= 2.9365, Training Accuracy= 0.600
Step 1000, Minibatch Loss= 1.8700, Training Accuracy= 0.660
Step 1100, Minibatch Loss= 1.8474, Training Accuracy= 0.640
Step 1200, Minibatch Loss= 0.7059, Training Accuracy= 0.730
Testing Accuracy: 0.56
Step 1300, Minibatch Loss= 1.3602, Training Accuracy= 0.670
Step 1400, Minibatch Loss= 1.0577, Training Accuracy= 0.740
Step 1500, Minibatch Loss= 0.4348, Training Accuracy= 0.870